In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


In [2]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.3 MB/s eta 0:00:00


In [3]:
import os
import json
#from dotenv import load_dotenv
from google.colab import userdata
from openai import OpenAI
import gradio as gr
from deep_translator import GoogleTranslator

In [4]:
openai_api_key = userdata.get('OPENAI-API-KEY')
google_api_key = userdata.get('GOOGLE-API-KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyA_


In [5]:
openai = OpenAI(api_key=openai_api_key)
gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [6]:
openai_model = "gpt-4o-mini"
gemini_model = "gemini-2.0-flash"


In [7]:
system_message = "You are a helpful assistant for an hotel called HOTELAI in Genoa. "
system_message += "You are courteous and you manage the reservations of the hotel. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [8]:
reservations = {"michele": "Reservation for two people from 07/05/2025 to 10/05/2025", "pippo": {"Reservation for five people from 07/05/2026 to 10/05/2026"}}

In [9]:
def get_reservations(reservation_name):
    print(f"Tool reservations called for {reservation_name}")
    name = reservation_name.lower()
    return reservations.get(name, "Unknown")

In [10]:
get_reservations('michele')

Tool reservations called for michele


'Reservation for two people from 07/05/2025 to 10/05/2025'

In [11]:
reservation_function = {
    "name": "get_reservations",
    "description": "Get the booking reservation of the hotel. For example 'I have a reservation at this hotel, can you give me the information of my reservation?'",
    "parameters": {
        "type": "object",
        "properties": {
            "reservation_name": {
                "type": "string",
                "description": "The person that has booked the hotel",
            },
        },
        "required": ["reservation_name"],
        "additionalProperties": False
    }
}

In [12]:
tools = [{"type": "function", "function": reservation_function}]

In [37]:
class ChatState:
  def __init__(self) -> None:
    self.source = 'auto'
    self.target = 'it'

  # def canTranslate(self, c):
  #   if c == 'yes':
  #     self.target = 'it'
  #     self.translate = True
  #   else:
  #     self.target = 'en'
  #     self.translate = False


chat_state = ChatState()

In [42]:
def chat(message, history):
    # Original chat processing
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = openai_model, messages = messages, tools=tools)
    response_text = response.choices[0].message.content

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message,chat_state)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=openai_model, messages=messages)
        response_text = response.choices[0].message.content

        translated_response = translate_message(response_text, chat_state.source, chat_state.target)

        return translated_response

    return response_text

In [43]:
def translate_message(text, source_lang, target_lang):
  if target_lang == "en":
    return text
  try:
      translator = GoogleTranslator(source=source_lang, target=target_lang)
      return translator.translate(text)
  except:
      return f"Translation error: {text}"

In [44]:
def handle_tool_call(message,chat_state):
  tool_call = message.tool_calls[0]
  arguments = json.loads(tool_call.function.arguments)
  name = arguments.get('reservation_name')
  reservation = get_reservations(name)
  #reservation = translate_message(reservation, chat_state.source, chat_state.target)
  response = {
      "role": "tool",
      "content": json.dumps({"reservation_name": name,"reservation": reservation}),
      "tool_call_id": tool_call.id
  }
  return response, name

In [45]:
gr.ChatInterface(fn=chat, type="messages").launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9c14ddd66c82024d76.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool reservations called for Michele
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9c14ddd66c82024d76.gradio.live
